In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
sys.path.append("..")

In [2]:
# %pip install -U pandas

In [3]:
name = "rollback_analy"
# name = "inspiration_analy"
analy = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)
# df.head(2)

In [4]:
df = analy.get_basic_confidence_intervals()

In [5]:
from abautomator.visualizer import visualizer

from bokeh.io import output_notebook
from bokeh.plotting import show

output_notebook()

Loading BokehJS ...

In [6]:
df = analy.get_basic_confidence_intervals()
df.head(10)

exp_cond                metric      mean       std    count  \
0       Tx    n_granted_contacts  0.192331  0.394141  20002.0   
1       Tx  pct_granted_contacts  0.192331  0.394141  20002.0   
2  Control    n_granted_contacts  0.187399  0.390241  20331.0   
3  Control  pct_granted_contacts  0.187399  0.390241  20331.0   
4       Tx       n_entered_phone  0.484752  0.499780  20002.0   
5       Tx     pct_entered_phone  0.484752  0.499780  20002.0   
6  Control       n_entered_phone  0.474005  0.499435  20331.0   
7  Control     pct_entered_phone  0.473956  0.499334  20331.0   

                      factor_label  upper_68_ci  lower_68_ci  upper_95_ci  \
0         (n_granted_contacts, Tx)     0.586472    -0.201811     0.980613   
1       (pct_granted_contacts, Tx)     0.586472    -0.201811     0.980613   
2    (n_granted_contacts, Control)     0.577639    -0.202842     0.967880   
3  (pct_granted_contacts, Control)     0.577639    -0.202842     0.967880   
4            (n_entered_phone, Tx)     0.984531    -0.015028     1.484311   
5          (pct_entered_phone, Tx)     0.984531    -0.015028     1.484311   
6       (n_entered_phone, Control)     0.973440    -0.025429     1.472874   
7     (pct_entered_phone, Control)     0.973290    -0.025378     1.472623   

   lower_95_ci  
0    -0.595952  
1    -0.595952  
2    -0.593083  
3    -0.593083  
4    -0.514808  
5    -0.514808  
6    -0.524864  
7    -0.524711

In [7]:
p = visualizer.get_figure(analy.get_basic_confidence_intervals())
show(p)

cleaning


In [22]:
import scipy.stats
import numpy as np


In [23]:
df = analy.get_abs_diff_confidence_intervals()
df["metric"] = df["metric"].str.replace("_", " ")
df["metric"] = df["metric"].str.title()
df["metric"] = df["metric"].str.replace("Pct", "%")
df["factor_label"] = list(zip(df["metric"], df["exp_cond"]))

In [24]:
df.head()

exp_cond              metric   tx_mean    tx_std  tx_count  \
0       Tx  N Granted Contacts  0.192331  0.394141   20002.0   
1       Tx  % Granted Contacts  0.192331  0.394141   20002.0   
2       Tx     N Entered Phone  0.484752  0.499780   20002.0   
3       Tx     % Entered Phone  0.484752  0.499780   20002.0   

               factor_label  ctrl_mean  ctrl_std  ctrl_count      mean  \
0  (N Granted Contacts, Tx)   0.187399  0.390241     20331.0  0.004932   
1  (% Granted Contacts, Tx)   0.187399  0.390241     20331.0  0.004932   
2     (N Entered Phone, Tx)   0.474005  0.499435     20331.0  0.010746   
3     (% Entered Phone, Tx)   0.473956  0.499334     20331.0  0.010795   

        std  upper_68_ci  lower_68_ci  upper_95_ci  lower_95_ci  
0  0.003906     0.008838     0.001026     0.012744    -0.002880  
1  0.003906     0.008838     0.001026     0.012744    -0.002880  
2  0.004976     0.015722     0.005771     0.020697     0.000795  
3  0.004975     0.015771     0.005820     0.020746     0.000845

In [25]:
df["t_calc"] = df["mean"]/df["std"]
df["t_crit"] = scipy.stats.t.ppf(q=1-.05/2, df=df["tx_count"] + df["ctrl_count"] - 2)
df["p_value"] = scipy.stats.t.sf(np.abs(df["t_calc"]), df["tx_count"] + df["ctrl_count"] - 2)*2  # two-sided pvalue = Prob(abs(t)>tt)
df.head()

exp_cond              metric   tx_mean    tx_std  tx_count  \
0       Tx  N Granted Contacts  0.192331  0.394141   20002.0   
1       Tx  % Granted Contacts  0.192331  0.394141   20002.0   
2       Tx     N Entered Phone  0.484752  0.499780   20002.0   
3       Tx     % Entered Phone  0.484752  0.499780   20002.0   

               factor_label  ctrl_mean  ctrl_std  ctrl_count      mean  \
0  (N Granted Contacts, Tx)   0.187399  0.390241     20331.0  0.004932   
1  (% Granted Contacts, Tx)   0.187399  0.390241     20331.0  0.004932   
2     (N Entered Phone, Tx)   0.474005  0.499435     20331.0  0.010746   
3     (% Entered Phone, Tx)   0.473956  0.499334     20331.0  0.010795   

        std  upper_68_ci  lower_68_ci  upper_95_ci  lower_95_ci    t_calc  \
0  0.003906     0.008838     0.001026     0.012744    -0.002880  1.262720   
1  0.003906     0.008838     0.001026     0.012744    -0.002880  1.262720   
2  0.004976     0.015722     0.005771     0.020697     0.000795  2.159808   
3  0.004975     0.015771     0.005820     0.020746     0.000845  2.169912   

     t_crit   p_value  
0  1.960023  0.206697  
1  1.960023  0.206697  
2  1.960023  0.030793  
3  1.960023  0.030019

In [15]:
p = visualizer.get_figure(analy.get_abs_diff_confidence_intervals())
show(p)

In [20]:
p = visualizer.get_figure(analy.get_rel_diff_confidence_intervals())
show(p)